In [ ]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime
import webbrowser

In [ ]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
filing_links = pd.read_excel(
    "/Users/fuadhassan/Desktop/BDC_RA/ARCC/ARCC__sec_filing_links.xlsx")

path = '/Users/fuadhassan/Desktop/BDC_RA/ARCC/ARCC_Investment.xlsx'
xls = pd.ExcelFile(path)
all_sheets = pd.read_excel(path, sheet_name=None)

In [ ]:
dataframes = {}
# Loop through each sheet and create a DataFrame in the dictionary
for sheet_name, sheet_df in all_sheets.items():
    dataframes[sheet_name.replace(' ', '_')] = sheet_df
for sheet_name, sheet_df in dataframes.items():
    print(f"DataFrame name: {sheet_name} : {sheet_df.shape}")

In [ ]:
date_columns = ['Filing date', 'Reporting date']
for col in date_columns:
    filing_links[col] = pd.to_datetime(filing_links[col], format='%Y-%m-%d')
for col in date_columns:
    filing_links[col] = filing_links[col].dt.strftime("%B %d %Y")
filing_links.head()

In [ ]:

matches = []  # To store rows of matching data

for file, frame in dataframes.items():
    if frame.shape[0] < 100:
        print(file.replace('_', ' '))
        matching_rows = filing_links[filing_links['Reporting date'].str.replace(
            ' ', '') == file.replace('_', '')]
        if not matching_rows.empty:
            matches.append(matching_rows)
            print(matching_rows['Filings URL'].values[0])
            # webbrowser.open(matching_rows['Filings URL'].values[0])
working_class = pd.concat(matches, ignore_index=True)

In [ ]:
def parse_and_trim(content, content_type):
    if content_type == 'HTML':
        soup = BeautifulSoup(content, 'html.parser')
    else:
        soup = BeautifulSoup(content, 'html.parser')

    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass

    for linebreak in soup.find_all('br'):
        linebreak.extract()
    for linebreak in soup.find_all('hr'):
        linebreak.extract()

    return soup

In [ ]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775023000036/arcc-20230630.htm'
# date = 'June 30, 2023'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
# date = 'March 31, 2019'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746911001575/a2202281z10-k.htm'
# # date = 'December 31, 2010'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465910055721/a10-17362_110q.htm'
# date = 'September 30, 2010'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# date = 'March 31, 2018'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465913080832/a13-19678_110q.htm'
date = 'September 30, 2013'

qtr_date = date

response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables(soup_content, qtr_date):
    consolidated_schedule_regex = re.compile(
        r'^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'

    master_table = None
    all_tags = soup_content.find_all(True)
    count = 0

    for tag in soup_content.find_all(text=consolidated_schedule_regex):
        # print(len(soup_content.find_all(text=consolidated_schedule_regex)))
        next_line_text = tag.next.text.strip()
        date_str = re.search(date_regex_pattern, next_line_text)

        if date_str is None:
            next_line_text = tag.find_next(
                text=re.compile(date_regex_pattern)).text
            date_str = re.search(date_regex_pattern, next_line_text)

        if date_str is None:
            next_line = tag.next.text.strip()
            date_str = re.search(date_regex_pattern, next_line)

        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)

            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                count += 1
                html_table = tag.find_next('table')
                while html_table:
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(
                        r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                    new_table = new_table.dropna(how='all', axis=0)

                    if master_table is None:
                        master_table = new_table
                    else:
                        master_table = pd.concat(
                            [master_table, new_table], ignore_index=True)
                    print("ran")

                    # if html_table.find(text=re.compile(r'Other than', re.IGNORECASE)):
                    #     print(html_table.find(text=re.compile(
                    #         r'Other than', re.IGNORECASE)))
                    #     break
                    # elif html_table.find_next(text='Derivative Instruments'):
                    #     print(html_table.find_next(
                    #         text='Derivative Instruments'))
                    #     break
                    # elif html_table.find_next(text=consolidated_schedule_regex):
                    #     next_line_text = tag.find_next(
                    #         text=re.compile(date_regex_pattern)).text
                    #     date_str2 = re.search(
                    #         date_regex_pattern, next_line_text)
                    #     if date_str2.group(1) == date_str:
                    #         break
                    print(html_table.find_previous_sibling())
                    html_table = html_table.find_next('table')

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    return master_table


master_table = extract_tables(content, qtr_date)
master_table.to_csv('example.csv')
master_table.to_excel('example.xlsx')

master_table

In [ ]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775023000036/arcc-20230630.htm'
# date = 'June 30, 2023'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
# date = 'March 31, 2019'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746914001349/a2218339z10-k.htm'
date = 'December 31, 2013'

# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# date = 'March 31, 2018'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465913080832/a13-19678_110q.htm'
# date = 'September 30, 2013'

qtr_date = date

response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables(soup_content, qtr_date):
    consolidated_schedule_regex = re.compile(
        r'^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'

    master_table = None
    all_tags = soup_content.find_all(True)
    count = 0

    for tag in soup_content.find_all(True):
        text = tag.get_text(strip=True)
        if re.fullmatch(date_regex_pattern2, text):
            date_str = re.search(date_regex_pattern1, text)
            print(tag)
            print(date_str)
            if date_str is not None:
                date_str = str(date_str.group(1))
                date_str = unicodedata.normalize('NFKD', date_str)

            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                html_table = tag.find_next('table')
                print('Table found: ')
                html_table = tag.find_next('table')
                if master_table is None:
                    master_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    master_table = master_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                              regex=True)
                    master_table = master_table.dropna(how='all', axis=0)
                else:
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                        regex=True)
                    new_table = new_table.dropna(how='all', axis=0)
                    # print('head')
                    # print(new_table.head()) # text
                    master_table = master_table.append(
                        new_table.dropna(how='all', axis=0).reset_index(
                            drop=True).drop(index=0),
                        ignore_index=True)

    return master_table


master_table = extract_tables(content, qtr_date)
# master_table.to_csv('example.csv')
# master_table.to_excel('example.xlsx')

master_table

In [ ]:
# this one works for all other
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775023000036/arcc-20230630.htm'
# date = 'June 30, 2023'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
# date = 'March 31, 2019'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746914001349/a2218339z10-k.htm'
date = 'December 31, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# date = 'March 31, 2018'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465913080832/a13-19678_110q.htm'
# date = 'September 30, 2013'

qtr_date = date

response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables(soup_content, qtr_date):
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    master_table = None

    for tag in soup_content.find_all(True):
        text = tag.get_text(strip=True)
        if re.fullmatch(date_regex_pattern2, text):
            date_str = re.search(date_regex_pattern1, text)
            next_line = tag.find_next().text
            if re.search('dollar', next_line):
                print(tag)
                print(True)
                print(date_str.group(1))
                if date_str is not None:
                    date_str = str(date_str.group(1))
                    date_str = unicodedata.normalize('NFKD', date_str)
                    # print(date_str)
                if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                    # print('match')
                    html_table = tag.find_next('table')
                    print('Table found: ')
                    html_table = tag.find_next('table')
                    if master_table is None:
                        master_table = pd.read_html(
                            html_table.prettify(), skiprows=0, flavor='bs4')[0]
                        master_table = master_table.applymap(lambda x: unicodedata.normalize(
                            'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                        master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                                  regex=True)
                        master_table = master_table.dropna(how='all', axis=0)
                    else:
                        new_table = pd.read_html(
                            html_table.prettify(), skiprows=0, flavor='bs4')[0]
                        new_table = new_table.applymap(lambda x: unicodedata.normalize(
                            'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                        new_table = new_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                            regex=True)
                        new_table = new_table.dropna(how='all', axis=0)
                        # print('head')
                        # print(new_table.head()) # text
                        master_table = master_table.append(
                            new_table.dropna(how='all', axis=0).reset_index(
                                drop=True).drop(index=0),
                            ignore_index=True)
    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    return master_table


master_table = extract_tables(content, qtr_date)
master_table.to_csv('example.csv')
master_table.to_excel('example.xlsx')
master_table

In [ ]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775023000036/arcc-20230630.htm'
# date = 'June 30, 2023'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
# date = 'March 31, 2019'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746914001349/a2218339z10-k.htm'
date = 'December 31, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# date = 'March 31, 2018'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465913080832/a13-19678_110q.htm'
# date = 'September 30, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000007/arccq4-1710k.htm'
# date = 'December 31, 2017'

qtr_date = date

response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables(soup_content, qtr_date):
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    master_table = None

    for tag in soup_content.find_all(text=re.compile(date_regex_pattern2)):

        date_str = re.search(date_regex_pattern1, tag.text)
        find_next = tag.find_next().text
        next_line = tag.next.text

        if re.search('dollar amounts', find_next) or re.search('dollar amounts', next_line):
            print(date_str.group(1))
            if date_str is not None:
                date_str = str(date_str.group(1))
                date_str = unicodedata.normalize('NFKD', date_str)
                # print(date_str)
            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                # print(tag)
                # print(find_next)
                # print(next_line)
                # print('match')
                html_table = tag.find_next('table')
                print('Table found: ')
                html_table = tag.find_next('table')
                if master_table is None:
                    master_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    master_table = master_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                              regex=True)
                    master_table = master_table.dropna(how='all', axis=0)
                else:
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                        regex=True)
                    new_table = new_table.dropna(how='all', axis=0)
                    # print('head')
                    # print(new_table.head()) # text
                    master_table = master_table.append(
                        new_table.dropna(how='all', axis=0).reset_index(
                            drop=True).drop(index=0),
                        ignore_index=True)
    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    return master_table


master_table = extract_tables(content, qtr_date)
master_table.to_csv('example.csv')
master_table.to_excel('example.xlsx')
# master_table

In [ ]:
def process_table(soi_table_df, append_str):
    soi_table_df = soi_table_df.replace(r'^\s*\$\s*$', np.nan, regex=True)
    soi_table_df = soi_table_df.dropna(how='all', axis=1)
    soi_table_df = soi_table_df.dropna(
        how='all', axis=0).reset_index(drop=True)
    # print('1: ' + str(soi_table_df.shape))

    # Separate header and data
    soi_table_header = soi_table_df.iloc[0].dropna(how='any')
    soi_table_data_df = soi_table_df.rename(
        columns=soi_table_df.iloc[0]).drop(soi_table_df.index[0])
    # print('2: ' + str(soi_table_data_df.shape))

    # drops all the rows that contains header
    soi_table_data_df = soi_table_data_df[soi_table_data_df[soi_table_data_df.columns[0]]
                                          != soi_table_data_df.columns[0]]
    # print('3: ' + str(soi_table_data_df.shape))

    # Drop rows with only two non-null values becuase all the subtotal contain 2 value only
    soi_table_data_df = soi_table_data_df.dropna(thresh=3)
    # print('4: ' + str(soi_table_data_df.shape))

    # replace all the - in the data table with 0
    soi_table_data_df = soi_table_data_df.replace('-', 0, regex=False)

    # fix the all the nan value column , Amortized Cost, Fair Value
    columns_to_fill = ['Amortized Cost', 'Fair Value']
    for col in columns_to_fill:
        col_index = soi_table_data_df.columns.str.replace(
            ' ', '').get_loc(col.replace(' ', ''))
        next_col_index = col_index + 1
        for i in range(len(soi_table_data_df)):
            current_value = soi_table_data_df.iat[i, col_index]
            if pd.isna(current_value) and next_col_index < len(soi_table_data_df.columns):
                next_valid_index = next((j for j, v in enumerate(
                    soi_table_data_df.iloc[i, next_col_index:], start=next_col_index) if pd.notna(v)), None)

                if next_valid_index is not None:
                    next_value = soi_table_data_df.iat[i, next_valid_index]
                    soi_table_data_df.iat[i, col_index] = next_value
                    soi_table_data_df.iat[i, next_valid_index] = pd.NA

    # drops everything after FairValue
    if 'FairValue' in soi_table_data_df.columns.str.replace(' ', ''):
        start_index = soi_table_data_df.columns.str.replace(
            ' ', '').get_loc('FairValue')
        soi_table_data_df = soi_table_data_df.iloc[:, :start_index+1]

    # Drop rows with only two non-null values this one recheacks
    soi_table_data_df = soi_table_data_df.dropna(thresh=3)

    # Forward fill the first two columns
    col_indices = [0, 1]
    soi_table_data_df.iloc[:, col_indices] = soi_table_data_df.iloc[:, col_indices].fillna(
        method='ffill')
    # print('7: ' + str(soi_table_data_df.shape))

    # Drop rows with all missing values
    soi_table_df = soi_table_df.dropna(how='all')
    # print('5: ' + str(soi_table_data_df.shape))

    # Drop columns with all missing values
    soi_table_data_df = soi_table_data_df.dropna(how='all', axis=1)
    # print('6: ' + str(soi_table_data_df.shape))

    # fill all the nan with 0
    soi_table_data_df = soi_table_data_df.fillna(0)

    # converts to number
    # cols_to_convert = ['Shares/Units', 'Principal',
    #                    'Amortized Cost', 'Fair Value']
    # for col in cols_to_convert:
    #     if col.replace(' ', '') in soi_table_data_df.columns.str.replace(' ', ''):
    #         col_index = soi_table_data_df.columns.str.replace(
    #             ' ', '').get_loc(col.replace(' ', ''))
    #         soi_table_data_df.iloc[:, col_index] = pd.to_numeric(
    #             soi_table_data_df.iloc[:, col_index], errors='coerce').fillna(0)

    cols_to_convert = ['Shares/Units', 'Principal',
                       'Amortized Cost', 'Fair Value']
    for col in cols_to_convert:
        if col.replace(' ', '') in soi_table_data_df.columns.str.replace(' ', ''):
            col_index = soi_table_data_df.columns.str.replace(
                ' ', '').get_loc(col.replace(' ', ''))
            converted_data = pd.to_numeric(
                soi_table_data_df.iloc[:, col_index], errors='coerce').fillna(0)
            soi_table_data_df[soi_table_data_df.columns[col_index]
                              ] = converted_data

    soi_table_data_df = soi_table_data_df.reset_index(drop=True)

    # keeping the total for now to check if the total is right
    # Drop rows based on regex pattern (e.g., 'subtotal' or 'total')
    # pattern = r'^([Ss]ubtotal)|([Tt]otal)'
    # mask = soi_table_data_df.apply(lambda row: row.astype(
    #     str).str.contains(pattern, case=False, na=False)).any(axis=1)
    # soi_table_data_df = soi_table_data_df[~mask]
    # # print('4: ' + str(soi_table_data_df.shape))

    # we dont need this here because we are dropping subtotal before
    # Drop rows labeled as subtotals
    # subtotal_rows = soi_table_data_df[soi_table_data_df['Company (1)'].str.replace(' ', '').str.contains(
    #     'subtotal', case=False, na=False)]
    # soi_table_data_df = soi_table_data_df[~soi_table_data_df.index.isin(
    #     subtotal_rows.index)]
    # print('3: ' + str(soi_table_data_df.shape))

    # we dont need this here because we are dropping subtotal before
    # columns_to_drop = []
    # for column in soi_table_data_df.columns:
    #     # Check for NaN values in the column
    #     # Use .item() to get a single boolean value
    #     if soi_table_data_df[column].isna().any().item():
    #         columns_to_drop.append(column)
    # soi_table_data_df.drop(columns=columns_to_drop, inplace=True)

    # print('Final: ' + str(soi_table_data_df.shape))

    return soi_table_data_df


process_table_ = process_table(master_table, "")

In [ ]:

headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775023000036/arcc-20230630.htm'
# date = 'June 30, 2023'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
# date = 'March 31, 2019'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746914001349/a2218339z10-k.htm'
date = 'December 31, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# date = 'March 31, 2018'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465913080832/a13-19678_110q.htm'
# date = 'September 30, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000007/arccq4-1710k.htm'
# date = 'December 31, 2017'

qtr_date = date

response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables(soup_content, qtr_date):
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    master_table = None

    for tag in soup_content.find_all(text=re.compile(date_regex_pattern2)):

        date_str = re.search(date_regex_pattern1, tag.text)
        find_next = tag.find_next().text
        next_line = tag.next.text
        if re.search('dollar amounts', find_next) or re.search('dollar amounts', next_line):
            print(date_str.group(1))
            if date_str is not None:
                date_str = str(date_str.group(1))
                date_str = unicodedata.normalize('NFKD', date_str)
                # print(date_str)
            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                # print(tag)
                # print(find_next)
                # print(next_line)
                # print('match')
                html_table = tag.find_next('table')
                print('Table found: ')
                # while soup_content.find_next(text=re.compile(date_regex_pattern2)) is not False:
                if master_table is None:
                    master_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    master_table = master_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                              regex=True)
                    master_table = master_table.dropna(how='all', axis=0)
                else:
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                        regex=True)
                    new_table = new_table.dropna(how='all', axis=0)
                    # print('head')
                    # print(new_table.head()) # text
                    master_table = master_table.append(
                        new_table.dropna(how='all', axis=0).reset_index(
                            drop=True).drop(index=0),
                        ignore_index=True)

                # while soup_content.find_next(text=re.compile(date_regex_pattern2)) is not False:
                # while html_table.find_next('table'):
                #     print(html_table)
                #     if soup_content.find(text=re.compile(date_regex_pattern2)):
                #         print('broke')
                #         break
                    html_table = html_table.find_next('table')

    # master_table = master_table.applymap(
    #     lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    # master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
    #     r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    # print(master_table.shape)
    return master_table


master_table = extract_tables(content, qtr_date)
# master_table.to_csv('example.csv')
# master_table.to_excel('example.xlsx')
# master_table